In [1]:
# import libraries

import os
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import mygene
import sklearn.preprocessing as sk
import seaborn as sns
from sklearn import metrics
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from torch.utils.data.sampler import WeightedRandomSampler
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score, precision_recall_curve
import random
import warnings
from random import randint
from sklearn.model_selection import StratifiedKFold

# Suppress all warnings
warnings.filterwarnings("ignore")

In [2]:
# define directories

cell_line_dir = "/Volumes/Expansion/Thesis Work/Supplementary Files/GDSC/"
models_dir = "/Volumes/Expansion/Thesis Work/Results/Transfer Learning/Drugs with same pathways/DNA_REPi/Expression_Mutation_CNA/Models/GDSC_PDX_TCGA_Second_Strategy/"
finetuned_dir = "/Volumes/Expansion/Thesis Work/Results/Transfer Learning/Drugs with same pathways/DNA_REPi/Expression_Mutation_CNA/Models/GDSC_PDX_TCGA_Second_Strategy/Second_Strategy_Finetuned_Models/"
DEGs_dir = "/Volumes/Expansion/Thesis Work/Results/GDSC_DEGs_inhibitors/DNA_REPi/"
dataset_dir = "/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/"
pdx_data_dir = "/Volumes/Expansion/Thesis Work/Datasets/PDX/Expression/"
save_results_to = "/Volumes/Expansion/Thesis Work/Results/Transfer Learning/Drugs with same pathways/DNA_REPi/Expression_Mutation_CNA/Predictions/GDSC_PDX_TCGA_Second_Strategy/"

In [3]:
# set random seeds

torch.manual_seed(42)
random.seed(42)

In [4]:
# change directory

os.chdir(cell_line_dir)
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Supplementary Files/GDSC


In [5]:
# read cell line details table in GDSC

GDSC_cell_line_details = pd.read_excel("GDSC_Cell_Lines_Details.xlsx", keep_default_na = False)
GDSC_cell_line_details.set_index("COSMIC identifier", inplace = True)
GDSC_cell_line_details.tail()

,Sample Name,Whole Exome Sequencing (WES),Copy Number Alterations (CNA),Gene Expression,Methylation,Drug\nResponse,GDSC\nTissue descriptor 1,GDSC\nTissue\ndescriptor 2,Cancer Type\n(matching TCGA label),Microsatellite \ninstability Status (MSI),Screen Medium,Growth Properties
COSMIC identifier,,,,,,,,,,,,
1298218,KP-2,Y,N,Y,N,Y,pancreas,pancreas,PAAD,,D/F12,Adherent
1330932,KO52,Y,Y,N,N,N,leukemia,acute_myeloid_leukaemia,LAML,MSS/MSI-L,D/F12,NA
1331030,SC-1,Y,Y,N,N,N,lymphoma,B_cell_lymphoma,DLBC,MSS/MSI-L,R,NA
1503373,U-CH2,Y,Y,N,N,N,bone,bone_other,,MSS/MSI-L,D/F12,Adherent
,TOTAL:,1001,996,968,957,990,,,,,,


In [6]:
# change directory to read GDSC expression dataset (DNA_REPi)

os.chdir(dataset_dir + "/exprs/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/exprs


In [7]:
# read GDSC expression dataset (DNA_REPi)

GDSCE = pd.read_csv("GDSC_exprs.z.DNA_REPi.tsv", 
                    sep = "\t", index_col=0, decimal = ",")
GDSCE = pd.DataFrame.transpose(GDSCE)
GDSCE.head(3)

ENTREZID,1,2,9,10,12,13,14,15,16,18,...,107984199,107984208,107984325,107984411,107984648,107984889,107984989,107986313,107986782,107986810
683665,-0.4711563767951986,-0.1857522957766964,0.9702409548662462,-0.4081281805960821,-0.4161511169999803,-0.4384159723467665,-1.154692246005183,0.5262132484107975,-0.7950924576033422,-0.6337442716864214,...,1.283534690590172,-0.3367265464586691,-0.0039762988723447,0.7378729872604209,0.9757614264575688,0.1494417647095801,2.062294063911948,-0.8140636761719959,0.2870333828833719,0.9524265085537148
684055,1.3565140948247052,-0.277542411913307,0.0887087882196536,-0.2754829982302413,-0.5153856346587746,-0.6248712076433911,0.1243408990866293,-1.2699667684674136,2.0788360886382717,2.5197991997459197,...,-0.5571028630154293,-1.724090733574364,-0.3749080365068901,-0.0016055431935905,-0.3788535663547365,-1.3502092798315848,0.6949790577550328,0.4333032198982747,-0.1280986270308099,-1.629249958712223
684057,0.6510004960254864,1.6578876382433665,-0.4977106390881842,-0.1212172092933644,-0.398455281697677,-0.2804901106672752,0.6455898116420014,0.215833091774836,-0.1723595788875009,-0.2170870770552223,...,-0.024863133395904,0.4073439017665206,-0.4440356638831038,0.8630347078663075,1.4486171428489227,-1.4777214830771732,0.5220400480464272,0.6441211958947753,0.9057518606131394,0.3018783753479829


In [8]:
# change directory to read GDSC mutation dataset (DNA_REPi)

os.chdir(dataset_dir + "/mutations/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/mutations


In [9]:
# read GDSC mutation dataset (DNA_REPi)

GDSCM = pd.read_csv("GDSC_mutations.DNA_REPi.tsv",
                    sep="\t", index_col=0, decimal=".")
GDSCM.drop_duplicates(keep='last')
GDSCM = pd.DataFrame.transpose(GDSCM)
GDSCM = GDSCM.loc[:, ~GDSCM.columns.duplicated()]

GDSCM.head()

,143872,728577,2,2050,10243,79365,256006,4609,118788,113675,...,26517,28972,53917,283507,514,339210,64981,7335,5739,83442
683665,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
684055,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
684057,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
684059,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
684062,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# change directory to read GDSC CNA dataset (DNA_REPi)

os.chdir(dataset_dir + "/CNA/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/CNA


In [11]:
# read GDSC CNA dataset (DNA_REPi)

GDSCC = pd.read_csv("GDSC_CNA.DNA_REPi.tsv",
                    sep="\t", index_col=0, decimal=".")
GDSCC.drop_duplicates(keep='last')
GDSCC = pd.DataFrame.transpose(GDSCC)
GDSCC = GDSCC.loc[:, ~GDSCC.columns.duplicated()]

In [12]:
# change directory to read GDSC response dataset (DNA_REPi)

os.chdir(dataset_dir + "/response/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/response


In [13]:
# read GDSC response dataset (DNA_REPi)

GDSCR = pd.read_csv("GDSC_response.DNA_REPi.tsv",
                    sep="\t",
                    index_col=0,
                    decimal=",")
GDSCR.dropna(inplace=True)
GDSCR.rename(mapper=str, axis='index', inplace=True)
d = {"R": 0, "S": 1}
GDSCR["response"] = GDSCR.loc[:, "response"].apply(lambda x: d[x])
GDSCR = GDSCR.loc[GDSCR["drug"] != "5-fluorouracil", ]

GDSCR.head()

,response,logIC50,drug,exprs,CNA,mutations
sample_name,,,,,,
683665,0,5.12075503641523,Pyrimethamine,1,1,1
683665,0,-3.03711635474651,Doxorubicin,1,1,1
683665,0,-0.712119156515401,Etoposide,1,1,1
683665,0,-4.40897202584936,Gemcitabine,1,1,1
683665,0,-1.59752443691324,Mitomycin C,1,1,1


In [14]:
# print count of cell lines for each drug (DNA_REPi)

GDSCR["drug"].value_counts()

drug
Etoposide         775
SN-38             771
Mitomycin C       767
Doxorubicin       766
5-Fluorouracil    764
Gemcitabine       761
Bleomycin         759
Temozolomide      752
Cisplatin         738
Methotrexate      738
Camptothecin      737
Cytarabine        735
Pyrimethamine     372
Name: count, dtype: int64

In [15]:
# change directory to read PDX and TCGA expression datasets homogenized with GDSC expression dataset (5-Fluorouracil)

os.chdir(dataset_dir + "exprs_homogenized/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/exprs_homogenized


In [16]:
# read PDX expression dataset homogenized with GDSC expression dataset (5-Fluorouracil)

PDXEfluorouracil = pd.read_csv("PDX_exprs.5-Fluorouracil.eb_with.GDSC_exprs.5-Fluorouracil.tsv",
                   sep = "\t", index_col=0, decimal = ",")
PDXEfluorouracil = pd.DataFrame.transpose(PDXEfluorouracil)
PDXEfluorouracil.head(3)

ENTREZID,1,2,9,10,12,13,14,15,16,18,...,100507206,100507254,100507436,100507472,100526773,100527978,100532746,100820829,102724473,105375355
X-1027,4.11649564280422,3.59209496158439,6.28559101273304,2.67926549063284,3.04621480878604,2.69044790660834,8.01420464403606,2.73140674359613,8.71768731731273,2.89488898093329,...,2.71033163066003,3.2543218456306,5.67560198356418,3.07143466694584,3.0597754133993,2.63635721806084,2.61292331163008,3.18623983789074,0,2.4522722382887
X-1119,4.00318976265845,3.59209496158439,7.55375516379688,4.03669966422185,3.41557516120483,5.94712898968845,8.14610109582064,2.54373640699005,7.8874359290969,3.43325748497387,...,2.80929358976047,3.3020335598854,8.94930072777643,3.07143466694584,3.0597754133993,2.63635721806084,2.61292331163008,4.60199637750889,0,2.89512503666773
X-1173,3.74198340066366,3.67493736139833,6.16750446793135,3.50545765081401,3.04295501121306,2.98779164904873,8.37697489020915,2.64942372799293,8.2967442081863,4.13640544224621,...,2.71033163066003,3.2543218456306,4.67270724878415,3.07143466694584,3.0597754133993,2.63635721806084,2.61292331163008,4.99706823903714,0,3.34491841640924


In [17]:
# read TCGA expression dataset homogenized with GDSC expression dataset (5-Fluorouracil)

TCGAEfluorouracil = pd.read_csv("TCGA_exprs.Fluorouracil.eb_with.GDSC_exprs.5-Fluorouracil.tsv",
                   sep = "\t", index_col=0, decimal = ",")
TCGAEfluorouracil = pd.DataFrame.transpose(TCGAEfluorouracil)
TCGAEfluorouracil.head(3)

ENTREZID,1,2,9,10,12,13,14,15,16,18,...,100188893,100190940,100190949,100191040,100192386,100268168,100271715,100287718,100288778,100289635
TCGA-3A-A9IC,5.72964960061116,4.41876362025309,4.86023990366878,2.47277876318537,4.81396331677419,2.6727941627861,8.72268461889477,2.5342747028707,8.30356599972612,2.66680810277045,...,9.61327758421809,2.69169602849574,2.74156299885291,3.14358416474115,3.63806017988108,2.83936659460498,2.84468471240513,2.9097524321588,4.74904572228446,4.63830114019886
TCGA-3A-A9IX,5.16820185705619,5.0656506669595,5.05015666335564,2.49748103264488,6.24486748738657,3.89184369807008,8.07246325142995,2.65471072107481,8.69196063245573,3.97461007293206,...,9.28039145658571,2.68523221938676,2.70770230320119,3.1831168236566,3.17601340812919,2.66266337139924,3.05740520413346,2.9097524321588,4.66674620630048,4.95684240309785
TCGA-AG-3593,4.09489404413517,3.56187701617794,5.74195136365074,3.52505514866919,0.688407410705727,2.04131612004455,8.50563761093224,2.83121226059543,8.98277420895832,4.7862922506412,...,9.44820752386781,3.3077275660044,3.04450667890368,3.53272687811095,2.90679846697165,2.45941479590968,2.65824917267036,2.9097524321588,5.80963066021836,4.86261821976041


In [18]:
# change directory to read PDX mutation dataset (5-Fluorouracil)

os.chdir(dataset_dir + "mutations")
os.getcwd()

'/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/mutations'

In [19]:
# read PDX mutation dataset (5-Fluorouracil)

PDXMfluorouracil = pd.read_csv("PDX_mutations.5-Fluorouracil.tsv",
                        sep="\t", index_col=0, decimal=",")
PDXMfluorouracil.drop_duplicates(keep='last')
PDXMfluorouracil = pd.DataFrame.transpose(PDXMfluorouracil)
PDXMfluorouracil = PDXMfluorouracil.loc[:, ~PDXMfluorouracil.columns.duplicated()]

PDXMfluorouracil.head(3)

ENTREZID,3075.0,171017.0,2058.0,79890.0,7701.0,3104.0,4642.0,9778.0,100132406.0,23607.0,...,253738.0,154807.0,79140.0,6307.0,6352.0,56005.0,115650.0,64946.0,5777.0,253639.0
X-1027,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
X-1119,0.0,-0.05,0.0,-0.05,0.0,0.0,0.0,0.0,-0.05,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
X-1173,0.0,0.0,0.0,-0.05,0.0,0.0,0.0,0.0,-0.05,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
# read TCGA mutation dataset (5-Fluorouracil)

TCGAMfluorouracil = pd.read_csv("TCGA_mutations.Fluorouracil.tsv",
                        sep="\t", index_col=0, decimal=",")
TCGAMfluorouracil.drop_duplicates(keep='last')
TCGAMfluorouracil = pd.DataFrame.transpose(TCGAMfluorouracil)
TCGAMfluorouracil = TCGAMfluorouracil.loc[:, ~TCGAMfluorouracil.columns.duplicated()]

TCGAMfluorouracil.head(3)

,9729,2050,515,140803,55814,51207,27143,152586,283659,51725,...,170393,401940,3249,254958,388199,55049,162699,114299,55969,147841
TCGA-3A-A9IC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-3A-A9IX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-AG-3593,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
# change directory to read PDX CNA dataset (5-Fluorouracil)

os.chdir(dataset_dir + "CNA")
os.getcwd()

'/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/CNA'

In [22]:
# read PDX CNA dataset (5-Fluorouracil)

PDXCfluorouracil = pd.read_csv("PDX_CNA.5-Fluorouracil.tsv",
                        sep="\t", index_col=0, decimal=",")
PDXCfluorouracil.drop_duplicates(keep='last')
PDXCfluorouracil = pd.DataFrame.transpose(PDXCfluorouracil)
PDXCfluorouracil = PDXCfluorouracil.loc[:, ~PDXCfluorouracil.columns.duplicated()]

PDXCfluorouracil.head(3)

ENTREZID,1,2,3,9,10,12,13,14,15,16,...,101340252,102723547,102724473,103091865,105375355,107126285,109623460,109731405,118126072,128385369
X-1027,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-1.1844245711374275,0.0,0.0,0.0,0.0
X-1119,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.7311832415721999,0.0,0.0,0.0,0.4276061727818994,0.0
X-1173,0.0,0.0,0.0,0.0,0.0,0.3391373849195853,0.0,0.3504972470841331,0.0,0.0,...,0.3391373849195853,0.3785116232537298,0.3673710656485294,0.0,0.8196681834964555,0.3617683594191533,0.0,0.0,0.3219280948873623,0.0


In [23]:
# read TCGA CNA dataset (5-Fluorouracil)

TCGACfluorouracil = pd.read_csv("TCGA_CNA.Fluorouracil.tsv",
                        sep="\t", index_col=0, decimal=",")
TCGACfluorouracil.drop_duplicates(keep='last')
TCGACfluorouracil = pd.DataFrame.transpose(TCGACfluorouracil)
TCGACfluorouracil = TCGACfluorouracil.loc[:, ~TCGACfluorouracil.columns.duplicated()]

TCGACfluorouracil.head(3)

,9,10,11,24,27,29,32,34,35,36,...,728851,100130391,100290877,100310869,100420569,100421288,100422928,100506036,100506076,100506123
TCGA-3A-A9IC,-0.2644,-0.2644,-0.2644,0.0,0.3214,-0.2518,-0.2497,0.0,-0.2497,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-3A-A9IX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-AG-3593,-0.7299,-0.7299,-0.7299,-0.3147,0.4133,-0.7013,-0.294,-0.3147,-0.3051,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
# variance threshold for GDSC expression dataset (DNA_REPi)

selector = VarianceThreshold(0.05)
selector.fit_transform(GDSCE)
GDSCE = GDSCE[GDSCE.columns[selector.get_support(indices=True)]]

In [25]:
# fill NA values and binarize GDSC CNA and mutation datasets (DNA_REPi)

GDSCM = GDSCM.fillna(0)
GDSCM[GDSCM != 0.0] = 1
GDSCC = GDSCC.fillna(0)
GDSCC[GDSCC != 0.0] = 1

In [26]:
# select shared genes between GDSC, PDX, and TCGA expression, mutation and CNA datasets 

ls = GDSCE.columns.intersection(GDSCM.columns)
ls = ls.intersection(GDSCC.columns)
ls = ls.intersection(PDXEfluorouracil.columns)
ls = ls.intersection(PDXMfluorouracil.columns)
ls = ls.intersection(PDXCfluorouracil.columns)
ls = ls.intersection(TCGAEfluorouracil.columns)
ls = ls.intersection(TCGAMfluorouracil.columns)
ls = ls.intersection(TCGACfluorouracil.columns)

ls = pd.unique(ls)

print(f"GDSC, PDX & TCGA # of common genes: {ls.shape[0]}")

GDSC, PDX & TCGA # of common genes: 12578


In [27]:
# select shared samples between GDSC expression, mutation, CNA, and response datasets (DNA_REPi)

ls2 = GDSCE.index.intersection(GDSCM.index)
ls2 = ls2.intersection(GDSCC.index)
ls2 = ls2.intersection(GDSCR.index)

print(f"GDSC # of common cell lines: {ls2.shape[0]}")

GDSC # of common cell lines: 801


In [28]:
# subset shared genes and samples in GDSC expression, mutation, CNA, and response datasets

GDSCE = GDSCE.loc[ls2, ls]
GDSCM = GDSCM.loc[ls2, ls]
GDSCC = GDSCC.loc[ls2, ls]
GDSCR = GDSCR.loc[ls2, :]

print(f"GDSC # of common samples and genes (Expression): {GDSCE.shape}")
print(f"GDSC # of common samples and genes (Mutation): {GDSCM.shape}")
print(f"GDSC # of common samples and genes (CNA): {GDSCC.shape}")
print(f"GDSC # of samples (Response): {GDSCR.shape[0]}")

GDSC # of common samples and genes (Expression): (801, 12578)
GDSC # of common samples and genes (Mutation): (801, 12578)
GDSC # of common samples and genes (CNA): (801, 12578)
GDSC # of samples (Response): 9435


In [29]:
# select shared samples between PDX expression, mutation, and CNA datasets (5-Fluorouracil) 

ls3 = PDXEfluorouracil.index.intersection(PDXMfluorouracil.index)
ls3 = ls3.intersection(PDXCfluorouracil.index)

print(f"PDX # of common samples (5-Fluorouracil): {ls3.shape[0]}")

PDX # of common samples (5-Fluorouracil): 23


In [30]:
# select shared samples between TCGA expression, mutation, and CNA datasets (5-Fluorouracil) 

ls4 = TCGAEfluorouracil.index.intersection(TCGAMfluorouracil.index)
ls4 = ls4.intersection(TCGACfluorouracil.index)

print(f"TCGA # of common samples (5-Fluorouracil): {ls4.shape[0]}")

TCGA # of common samples (5-Fluorouracil): 33


In [31]:
# select shared genes and samples between PDX and TCGA expression, mutation, and CNA datasets (5-Fluorouracil)

PDXEfluorouracil = PDXEfluorouracil.loc[ls3, ls]
PDXMfluorouracil = PDXMfluorouracil.loc[ls3, ls]
PDXCfluorouracil = PDXCfluorouracil.loc[ls3, ls]

TCGAEfluorouracil = TCGAEfluorouracil.loc[ls4, ls]
TCGAMfluorouracil = TCGAMfluorouracil.loc[ls4, ls]
TCGACfluorouracil = TCGACfluorouracil.loc[ls4, ls]

print(f"PDX # of common samples and genes for Expression (5-Fluorouracil): {PDXEfluorouracil.shape}")
print(f"PDX # of common samples and genes for Mutation (5-Fluorouracil): {PDXMfluorouracil.shape}")
print(f"PDX # of common samples and genes for CNA (5-Fluorouracil): {PDXCfluorouracil.shape}\n")

print(f"TCGA # of common samples and genes for Expression (5-Fluorouracil): {TCGAEfluorouracil.shape}")
print(f"TCGA # of common samples and genes for Mutation (5-Fluorouracil): {TCGAMfluorouracil.shape}")
print(f"TCGA # of common samples and genes for CNA (5-Fluorouracil): {TCGACfluorouracil.shape}")

PDX # of common samples and genes for Expression (5-Fluorouracil): (23, 12578)
PDX # of common samples and genes for Mutation (5-Fluorouracil): (23, 12578)
PDX # of common samples and genes for CNA (5-Fluorouracil): (23, 12578)

TCGA # of common samples and genes for Expression (5-Fluorouracil): (33, 12578)
TCGA # of common samples and genes for Mutation (5-Fluorouracil): (33, 12578)
TCGA # of common samples and genes for CNA (5-Fluorouracil): (33, 12578)


In [32]:
# change directory

os.chdir(DEGs_dir)
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/GDSC_DEGs_inhibitors/DNA_REPi


In [33]:
# read DNA_REPi_Differentially_Expressed_Genes common among DNA replication inhibitors

DEGs_filtered_data = pd.read_excel("DNA_REPi_Differentially_Expressed_Genes (EnsemblID).xlsx",
                                    sheet_name = "Common DEGs")
print(f"There are {DEGs_filtered_data.shape[0]} DEGs (p-adjusted < 0.05 and |logFC| > 1) among DNA replication inhibitors\n")

DEGs_filtered_data.head()

There are 858 DEGs (p-adjusted < 0.05 and |logFC| > 1) among DNA replication inhibitors



,Gene Symbol,Frequency
0,SLFN11,9
1,NCKAP1L,8
2,SDC4,7
3,KRT18,7
4,S100A14,7


In [34]:
filter = DEGs_filtered_data["Frequency"] == 1
DEGs_freq_one =  DEGs_filtered_data[filter]
DEGs_freq_one.head()

,Gene Symbol,Frequency
545,ELF1,1
546,PROSER2,1
547,CLIP4,1
548,THAP10,1
549,SHTN1,1


In [35]:
DEGs_fluorouracil_data = pd.read_excel("DNA_REPi_Differentially_Expressed_Genes (EnsemblID).xlsx",
                                              sheet_name = "5-Fluorouracil")

filter = DEGs_fluorouracil_data["Gene.Symbol"].isin(DEGs_freq_one["Gene Symbol"])
only_fluorouracil_degs = DEGs_fluorouracil_data.loc[filter, "Gene.Symbol"]

print(f'There are {only_fluorouracil_degs.shape[0]} DEGs unique to 5-FU, so they were removed from the DEGs list')

There are 24 DEGs unique to 5-FU, so they were removed from the DEGs list


In [36]:
filter = DEGs_filtered_data["Gene Symbol"].isin(only_fluorouracil_degs)
DEGs_filtered_data = DEGs_filtered_data[~filter]

DEGs_filtered_data.head()

,Gene Symbol,Frequency
0,SLFN11,9
1,NCKAP1L,8
2,SDC4,7
3,KRT18,7
4,S100A14,7


In [37]:
# get gene symbol from mygene database

mg = mygene.MyGeneInfo()
DEGs_entrez_id = mg.querymany(DEGs_filtered_data["Gene Symbol"], 
                              species="human", 
                              scopes = "symbol",
                              field = "entrezgene",
                              as_dataframe=True)["entrezgene"]

DEGs_entrez_id.dropna(inplace=True)

DEGs_entrez_id.shape[0]

1 input query terms found dup hits:	[('CLECL1P', 2)]


834

In [38]:
# assign GDSC datasets to new variables

exprs_z = GDSCE
mut = GDSCM
cna = GDSCC
responses = GDSCR

In [39]:
# GDSC response dataset

responses.head(3)

,response,logIC50,drug,exprs,CNA,mutations
683665,0,5.12075503641523,Pyrimethamine,1,1,1
683665,0,-3.03711635474651,Doxorubicin,1,1,1
683665,0,-0.712119156515401,Etoposide,1,1,1


In [40]:
# list drug names in GDSC response dataset

drugs = set(responses["drug"].values)
drugs

{'5-Fluorouracil',
 'Bleomycin',
 'Camptothecin',
 'Cisplatin',
 'Cytarabine',
 'Doxorubicin',
 'Etoposide',
 'Gemcitabine',
 'Methotrexate',
 'Mitomycin C',
 'Pyrimethamine',
 'SN-38',
 'Temozolomide'}

In [41]:
# convert Entrez gene IDs to integer

GDSCE.index = GDSCE.index.astype(str)
GDSCM.index = GDSCM.index.astype(str)
GDSCC.index = GDSCC.index.astype(str)
responses.index = responses.index.astype(str)

In [42]:
# merge GDSC cell line details with response dataset

dat = pd.merge(GDSC_cell_line_details,
         responses,
         left_index = True, 
         right_index=True,
         how = "inner")

dat["GDSC\nTissue descriptor 1"].value_counts()
#dat.head()

Series([], Name: count, dtype: int64)

In [43]:
# get tissue descriptors

dat["GDSC\nTissue\ndescriptor 2"].value_counts().head()

Series([], Name: count, dtype: int64)

In [44]:
# list count of cell lines as to tissues treated with fluorouracil

filter = (dat["drug"] == "5-Fluorouracil")
dat.loc[filter,"GDSC\nTissue descriptor 1"].value_counts()

Series([], Name: count, dtype: int64)

In [45]:
# convert Entrez gene IDs to string

responses.index = responses.index.astype(str)
responses.index

Index(['683665', '683665', '683665', '683665', '683665', '683665', '683665',
       '683665', '683665', '683665',
       ...
       '1659818', '1659818', '1659818', '1659818', '1659819', '1659819',
       '1659819', '1659819', '1659819', '687800'],
      dtype='object', length=9435)

In [46]:
# filter GDSC expression, mutation, and CNA datasets (DNA_REPi) as to drugs
# subset selected DEGs

expression_zscores = []
CNA = []
mutations = []
for drug in drugs:
    samples = responses.loc[responses["drug"] == drug, :].index.values
    e_z = exprs_z.loc[samples, :]
    c = cna.loc[samples, :]
    m = mut.loc[samples, :]

    expression_zscores.append(e_z)
    CNA.append(c)
    mutations.append(m)

GDSCEv2 = pd.concat(expression_zscores, axis=0)
GDSCMv2 = pd.concat(mutations, axis=0)
GDSCCv2 = pd.concat(CNA, axis=0)
GDSCRv2 = responses

ls5 = list(set(GDSCE.columns).intersection(set(DEGs_entrez_id.astype(int))))
ls6 = GDSCEv2.index.intersection(GDSCMv2.index)
ls6 = ls6.intersection(GDSCCv2.index)

GDSCEv2 = GDSCEv2.loc[ls6, ls5]
GDSCMv2 = GDSCMv2.loc[ls6, ls5]
GDSCCv2 = GDSCCv2.loc[ls6, ls5]
GDSCRv2 = GDSCRv2.loc[ls6, :]

PDXEfluorouracil = PDXEfluorouracil.loc[:,ls5]
PDXMfluorouracil = PDXMfluorouracil.loc[:,ls5]
PDXCfluorouracil = PDXCfluorouracil.loc[:,ls5]

TCGAEfluorouracil = TCGAEfluorouracil.loc[:,ls5]
TCGAMfluorouracil = TCGAMfluorouracil.loc[:,ls5]
TCGACfluorouracil = TCGACfluorouracil.loc[:,ls5]

responses.index = responses.index.values + "_" + responses["drug"].values

print(f"GDSC # of common samples and genes (Expression): {GDSCEv2.shape}")
print(f"GDSC # of common samples and genes (Mutation): {GDSCMv2.shape}")
print(f"GDSC # of common samples and genes (CNA): {GDSCCv2.shape}")
print(f"GDSC # of common samples (Response): {GDSCRv2.shape}\n")

print(f"PDX # of common samples and genes for 5-Fluorouracil (Expression): {PDXEfluorouracil.shape}")
print(f"PDX # of common samples and genes for 5-Fluorouracil (Mutation): {PDXMfluorouracil.shape}")
print(f"PDX # of common samples and genes for 5-Fluorouracil (CNA): {PDXCfluorouracil.shape}\n")

print(f"TCGA # of common samples and genes for 5-Fluorouracil (Expression): {TCGAEfluorouracil.shape}")
print(f"TCGA # of common samples and genes for 5-Fluorouracil (Mutation): {TCGAMfluorouracil.shape}")
print(f"TCGA # of common samples and genes for 5-Fluorouracil (CNA): {TCGACfluorouracil.shape}")

GDSC # of common samples and genes (Expression): (9435, 631)
GDSC # of common samples and genes (Mutation): (9435, 631)
GDSC # of common samples and genes (CNA): (9435, 631)
GDSC # of common samples (Response): (9435, 6)

PDX # of common samples and genes for 5-Fluorouracil (Expression): (23, 631)
PDX # of common samples and genes for 5-Fluorouracil (Mutation): (23, 631)
PDX # of common samples and genes for 5-Fluorouracil (CNA): (23, 631)

TCGA # of common samples and genes for 5-Fluorouracil (Expression): (33, 631)
TCGA # of common samples and genes for 5-Fluorouracil (Mutation): (33, 631)
TCGA # of common samples and genes for 5-Fluorouracil (CNA): (33, 631)


In [47]:
# GDSC response values

Y = GDSCRv2['response'].values
print(Y)

[0 0 0 ... 0 1 0]


In [48]:
# change directory to read PDX and TCGA response datasets (5-Fluorouracil)

os.chdir(dataset_dir + "response")
os.getcwd()

'/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/response'

In [49]:
# read PDX response dataset (5-Fluorouracil)

PDXRfluorouracil = pd.read_csv("PDX_response.5-Fluorouracil.tsv", 
                      sep = "\t", 
                      index_col=0, 
                      decimal = ",")
PDXRfluorouracil.dropna(inplace=True)
PDXRfluorouracil.rename(mapper=str, axis='index', inplace=True)
d = {"R": 0, "S": 1}
PDXRfluorouracil["response"] = PDXRfluorouracil.loc[:, "response"].apply(lambda x: d[x])
Ytsfluorouracil_pdx = PDXRfluorouracil["response"].values    

print(f"There are {Ytsfluorouracil_pdx.shape[0]} samples with response data in the PDX (5-Fluorouracil)\n")
Ytsfluorouracil_pdx

There are 23 samples with response data in the PDX (5-Fluorouracil)



array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0])

In [50]:
# read PDX dataset to select cohorts (5-Fluorouracil)

pdx_cohort = pd.read_excel(pdx_data_dir + "/nm.3954-S2.xlsx",
                              "PCT raw data")
pdx_cohort = pdx_cohort.loc[pdx_cohort["Treatment"] == "5FU",]
pdx_cohort = pdx_cohort.loc[:,["Model","Tumor Type"]]
pdx_cohort = pdx_cohort.drop_duplicates()
pdx_cohort.set_index("Model", inplace = True) 
pdx_cohort.head()

,Tumor Type
Model,
X-0933,CRC
X-1027,CRC
X-1055,CRC
X-1119,CRC
X-1167,CRC


In [51]:
# merged PDX cohort table with PDX response dataset (5-Fluorouracil)

dat_pdx = pd.merge(pdx_cohort,
                   PDXRfluorouracil,
                   left_index = True,
                   right_index = True)
dat_pdx["Tumor Type"].value_counts()

Tumor Type
CRC    23
Name: count, dtype: int64

In [52]:
# filter CRC cohort samples (5-Fluorouracil)

filter = (dat_pdx["Tumor Type"] == "CRC")
pdx_crc_samples = dat_pdx.loc[filter,].index
pdx_crc_samples

Index(['X-1027', 'X-1119', 'X-1173', 'X-1270', 'X-1329', 'X-2145', 'X-2182',
       'X-2483', 'X-2538', 'X-2573', 'X-2659', 'X-2846', 'X-2861', 'X-3093',
       'X-3205', 'X-3267', 'X-3671', 'X-3792', 'X-4087', 'X-5405', 'X-5438',
       'X-5495', 'X-5578'],
      dtype='object')

In [53]:
# filter responses of CRC cohort samples (5-Fluorouracil)

Ytsfluorouracil_pdx_crc = PDXRfluorouracil.loc[pdx_crc_samples,"response"].values
Ytsfluorouracil_pdx_crc

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0])

In [54]:
# count of sensitive and resistant CRC samples (5-Fluorouracil)

PDXRfluorouracil.loc[pdx_crc_samples,"response"].value_counts()

response
0    22
1     1
Name: count, dtype: int64

In [55]:
# read TCGA response dataset (5-Fluorouracil)

TCGARfluorouracil = pd.read_csv("TCGA_response.Fluorouracil.tsv", 
                      sep = "\t", 
                      index_col=0, 
                      decimal = ",")
TCGARfluorouracil.dropna(inplace=True)
TCGARfluorouracil.rename(mapper=str, axis='index', inplace=True)
d = {"R": 0, "S": 1}
TCGARfluorouracil["response"] = TCGARfluorouracil.loc[:, "response"].apply(lambda x: d[x])
Ytsfluorouracil_tcga = TCGARfluorouracil["response"].values    

print(f"There are {Ytsfluorouracil_tcga.shape[0]} samples with response data in the TCGA (5-Fluorouracil)\n")
Ytsfluorouracil_tcga

There are 33 samples with response data in the TCGA (5-Fluorouracil)



array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1])

In [56]:
# TCGA cohort table with TCGA response dataset (5-Fluorouracil)

dat_tcga = TCGARfluorouracil["cohort"]

dat_tcga.value_counts()

cohort
STAD    18
READ     7
PAAD     6
ESCA     2
Name: count, dtype: int64

In [57]:
# filter STAD cohort samples (5-Fluorouracil)

filter = (TCGARfluorouracil["cohort"] == "STAD")
tcga_stad_samples = dat_tcga.loc[filter,].index
tcga_stad_samples

Index(['TCGA-CG-4444', 'TCGA-D7-8579', 'TCGA-D7-A748', 'TCGA-F1-A448',
       'TCGA-FP-7829', 'TCGA-FP-8209', 'TCGA-HU-8243', 'TCGA-KB-A6F7',
       'TCGA-VQ-A8DT', 'TCGA-VQ-A8DU', 'TCGA-VQ-A8DZ', 'TCGA-VQ-A8E2',
       'TCGA-VQ-A8P3', 'TCGA-VQ-A91Q', 'TCGA-VQ-AA68', 'TCGA-VQ-AA6B',
       'TCGA-VQ-AA6F', 'TCGA-VQ-AA6G'],
      dtype='object')

In [58]:
# filter responses of STAD cohort samples (5-Fluorouracil)

Ytsfluorouracil_tcga_stad = TCGARfluorouracil.loc[tcga_stad_samples,"response"].values
Ytsfluorouracil_tcga_stad

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0])

In [59]:
# count of sensitive and resistance STAD samples (5-Fluorouracil)

TCGARfluorouracil.loc[tcga_stad_samples,"response"].value_counts()

response
1    13
0     5
Name: count, dtype: int64

In [60]:
# filter PAAD cohort samples (5-Fluorouracil)

filter = (TCGARfluorouracil["cohort"] == "PAAD")
tcga_paad_samples = dat_tcga.loc[filter,].index
tcga_paad_samples

Index(['TCGA-3A-A9IC', 'TCGA-3A-A9IX', 'TCGA-HZ-7920', 'TCGA-HZ-A49I',
       'TCGA-IB-7888', 'TCGA-OE-A75W'],
      dtype='object')

In [61]:
# filter responses of PAAD cohort samples (5-Fluorouracil)

Ytsfluorouracil_tcga_paad = TCGARfluorouracil.loc[tcga_paad_samples,"response"].values
Ytsfluorouracil_tcga_paad

array([0, 1, 1, 0, 0, 0])

In [62]:
# count of sensitive and resistance PAAD samples (5-Fluorouracil)

TCGARfluorouracil.loc[tcga_paad_samples,"response"].value_counts()

response
0    4
1    2
Name: count, dtype: int64

In [63]:
# maximum iteration

max_iter = 50

In [71]:
# use count of nodes and dropout rates found in trained pan-drug dataset

hdm1 = 128
hdm2 = 1024
hdm3 = 256
rate1 = 0.5
rate2 = 0.7
rate3 = 0.4
rate4 = 0.3

In [80]:
# load pre-trained modules and make predictions

scalerGDSC = sk.StandardScaler()
scalerGDSC.fit(GDSCEv2.values)
X_trainE = scalerGDSC.transform(GDSCEv2.values)
X_testEfluorouracil_pdx = scalerGDSC.transform(PDXEfluorouracil.values) 
X_testEfluorouracil_pdx_crc = scalerGDSC.transform(PDXEfluorouracil.loc[pdx_crc_samples,].values) 

X_testEfluorouracil_tcga = scalerGDSC.transform(TCGAEfluorouracil.values)    
X_testEfluorouracil_tcga_stad = scalerGDSC.transform(TCGAEfluorouracil.loc[tcga_stad_samples,].values)    
X_testEfluorouracil_tcga_paad = scalerGDSC.transform(TCGAEfluorouracil.loc[tcga_paad_samples,].values)    

X_trainM = np.nan_to_num(GDSCMv2.values)
X_testMfluorouracil_pdx = np.nan_to_num(PDXMfluorouracil.values.astype(np.float32))
X_testMfluorouracil_pdx_crc = np.nan_to_num(PDXMfluorouracil.loc[pdx_crc_samples,].values.astype(np.float32))
X_testMfluorouracil_tcga = np.nan_to_num(TCGAMfluorouracil.values.astype(np.float32))
X_testMfluorouracil_tcga_stad = np.nan_to_num(TCGAMfluorouracil.loc[tcga_stad_samples,].values)
X_testMfluorouracil_tcga_paad = np.nan_to_num(TCGAMfluorouracil.loc[tcga_paad_samples,].values.astype(np.float32))

X_trainC = np.nan_to_num(GDSCCv2.values)
X_testCfluorouracil_pdx = np.nan_to_num(PDXCfluorouracil.values.astype(np.float32))
X_testCfluorouracil_pdx_crc = np.nan_to_num(PDXCfluorouracil.loc[pdx_crc_samples,].values.astype(np.float32))
X_testCfluorouracil_tcga = np.nan_to_num(TCGACfluorouracil.values)
X_testCfluorouracil_tcga_stad = np.nan_to_num(TCGACfluorouracil.loc[tcga_stad_samples,].values)
X_testCfluorouracil_tcga_paad = np.nan_to_num(TCGACfluorouracil.loc[tcga_paad_samples,].values)

TX_testEfluorouracil_pdx = torch.FloatTensor(X_testEfluorouracil_pdx)
TX_testEfluorouracil_pdx_crc = torch.FloatTensor(X_testEfluorouracil_pdx_crc)

TX_testMfluorouracil_pdx = torch.FloatTensor(X_testMfluorouracil_pdx.astype(np.float32))
TX_testMfluorouracil_pdx_crc = torch.FloatTensor(X_testMfluorouracil_pdx_crc.astype(np.float32))

TX_testCfluorouracil_pdx = torch.FloatTensor(X_testCfluorouracil_pdx.astype(np.float32))
TX_testCfluorouracil_pdx_crc = torch.FloatTensor(X_testCfluorouracil_pdx_crc.astype(np.float32))

ty_testEfluorouracil_pdx = torch.FloatTensor(Ytsfluorouracil_pdx.astype(int))
ty_testEfluorouracil_pdx_crc = torch.FloatTensor(Ytsfluorouracil_pdx_crc.astype(int))

TX_testEfluorouracil_tcga = torch.FloatTensor(X_testEfluorouracil_tcga)
TX_testEfluorouracil_tcga_stad = torch.FloatTensor(X_testEfluorouracil_tcga_stad)
TX_testEfluorouracil_tcga_paad = torch.FloatTensor(X_testEfluorouracil_tcga_paad)

TX_testMfluorouracil_tcga = torch.FloatTensor(X_testMfluorouracil_tcga.astype(np.float32))
TX_testMfluorouracil_tcga_stad = torch.FloatTensor(X_testMfluorouracil_tcga_stad.astype(np.float32))
TX_testMfluorouracil_tcga_paad = torch.FloatTensor(X_testMfluorouracil_tcga_paad.astype(np.float32))

TX_testCfluorouracil_tcga = torch.FloatTensor(X_testCfluorouracil_tcga.astype(np.float32))
TX_testCfluorouracil_tcga_stad = torch.FloatTensor(X_testCfluorouracil_tcga_stad.astype(np.float32))
TX_testCfluorouracil_tcga_paad = torch.FloatTensor(X_testCfluorouracil_tcga_paad.astype(np.float32))

ty_testEfluorouracil_tcga = torch.FloatTensor(Ytsfluorouracil_tcga.astype(int))
ty_testEfluorouracil_tcga_stad = torch.FloatTensor(Ytsfluorouracil_tcga_stad.astype(int))
ty_testEfluorouracil_tcga_paad = torch.FloatTensor(Ytsfluorouracil_tcga_paad.astype(int))

n_sampE, IE_dim = X_trainE.shape
n_sampM, IM_dim = X_trainM.shape
n_sampC, IC_dim = X_trainC.shape

h_dim1 = hdm1
h_dim2 = hdm2
h_dim3 = hdm3        
Z_in = h_dim1 + h_dim2 + h_dim3

class AEE(nn.Module):
    def __init__(self):
        super(AEE, self).__init__()
        self.EnE = torch.nn.Sequential(
            nn.Linear(IE_dim, h_dim1),
            nn.BatchNorm1d(h_dim1),
            nn.ReLU(),
            nn.Dropout(rate1))
    def forward(self, x):
        output = self.EnE(x)
        return output  


class AEM(nn.Module):
    def __init__(self):
        super(AEM, self).__init__()
        self.EnM = torch.nn.Sequential(
            nn.Linear(IM_dim, h_dim2),
            nn.BatchNorm1d(h_dim2),
            nn.ReLU(),
            nn.Dropout(rate2))
    def forward(self, x):
        output = self.EnM(x)
        return output       

class AEC(nn.Module):
    def __init__(self):
        super(AEC, self).__init__()
        self.EnC = torch.nn.Sequential(
            nn.Linear(IC_dim, h_dim3),
            nn.BatchNorm1d(h_dim3),
            nn.ReLU(),
            nn.Dropout(rate3))
    def forward(self, x):
        output = self.EnC(x)
        return output 
        
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.FC = torch.nn.Sequential(
            nn.Linear(Z_in, 1),
            nn.Dropout(rate4),
            nn.Sigmoid())
    def forward(self, x):
        return self.FC(x)

torch.cuda.manual_seed_all(42)

AutoencoderE = torch.load(models_dir + "Exprs_Fluorouracil_GDSC_PDX_TCGA_DNA_REPi_Expression_Mutation_CNA_Second_Strategy.pt")
AutoencoderM = torch.load(models_dir + "Mut_Fluorouracil_GDSC_PDX_TCGA_DNA_REPi_Expression_Mutation_CNA_Second_Strategy.pt")
AutoencoderC = torch.load(models_dir + "CNA_Fluorouracil_GDSC_PDX_TCGA_DNA_REPi_Expression_Mutation_CNA_Second_Strategy.pt")

Clas = torch.load(models_dir + "Class_Fluorouracil_GDSC_PDX_TCGA_DNA_REPi_Expression_Mutation_CNA_Second_Strategy.pt")

AutoencoderE.eval()
AutoencoderM.eval()
AutoencoderC.eval()
Clas.eval()

ZEX = AutoencoderE(torch.FloatTensor(X_trainE))
ZMX = AutoencoderM(torch.FloatTensor(X_trainM))
ZCX = AutoencoderC(torch.FloatTensor(X_trainC))
ZTX = torch.cat((ZEX, ZMX, ZCX), 1)
ZTX = F.normalize(ZTX, p=2, dim=0)
PredX = Clas(ZTX)
AUCT = roc_auc_score(Y, PredX.detach().numpy())
AUCTaucpr = average_precision_score(Y, PredX.detach().numpy())

ZETfluorouracil_pdx = AutoencoderE(TX_testEfluorouracil_pdx)
ZMTfluorouracil_pdx = AutoencoderM(TX_testMfluorouracil_pdx)
ZCTfluorouracil_pdx = AutoencoderC(TX_testCfluorouracil_pdx)
ZTTfluorouracil_pdx = torch.cat((ZETfluorouracil_pdx, ZMTfluorouracil_pdx, ZCTfluorouracil_pdx), 1)
ZTTfluorouracil_pdx = F.normalize(ZTTfluorouracil_pdx, p=2, dim=0)
PredTfluorouracil_pdx = Clas(ZTTfluorouracil_pdx)
AUCTfluorouracil_pdx = roc_auc_score(Ytsfluorouracil_pdx.astype(int), PredTfluorouracil_pdx.detach().numpy())
AUCTfluorouracil_pdx_aucpr = average_precision_score(Ytsfluorouracil_pdx.astype(int), PredTfluorouracil_pdx.detach().numpy())

ZETfluorouracil_pdx_crc = AutoencoderE(TX_testEfluorouracil_pdx_crc)
ZMTfluorouracil_pdx_crc = AutoencoderM(TX_testMfluorouracil_pdx_crc)
ZCTfluorouracil_pdx_crc = AutoencoderC(TX_testCfluorouracil_pdx_crc)
ZTTfluorouracil_pdx_crc = torch.cat((ZETfluorouracil_pdx_crc, ZMTfluorouracil_pdx_crc, ZCTfluorouracil_pdx_crc), 1)
ZTTfluorouracil_pdx_crc = F.normalize(ZTTfluorouracil_pdx_crc, p=2, dim=0)
PredTfluorouracil_pdx_crc = Clas(ZTTfluorouracil_pdx_crc)
AUCTfluorouracil_pdx_crc = roc_auc_score(Ytsfluorouracil_pdx_crc.astype(int), PredTfluorouracil_pdx_crc.detach().numpy())
AUCTfluorouracil_pdx_crc_aucpr = average_precision_score(Ytsfluorouracil_pdx_crc.astype(int), PredTfluorouracil_pdx_crc.detach().numpy())

ZETfluorouracil_tcga = AutoencoderE(TX_testEfluorouracil_tcga)
ZMTfluorouracil_tcga = AutoencoderM(TX_testMfluorouracil_tcga)
ZCTfluorouracil_tcga = AutoencoderC(TX_testCfluorouracil_tcga)
ZTTfluorouracil_tcga = torch.cat((ZETfluorouracil_tcga, ZMTfluorouracil_tcga, ZCTfluorouracil_tcga), 1)
ZTTfluorouracil_tcga = F.normalize(ZTTfluorouracil_tcga, p=2, dim=0)
PredTfluorouracil_tcga = Clas(ZTTfluorouracil_tcga)
AUCTfluorouracil_tcga = roc_auc_score(Ytsfluorouracil_tcga.astype(int), PredTfluorouracil_tcga.detach().numpy())
AUCTfluorouracil_tcga_aucpr = average_precision_score(Ytsfluorouracil_tcga.astype(int), PredTfluorouracil_tcga.detach().numpy())

ZETfluorouracil_tcga_stad = AutoencoderE(TX_testEfluorouracil_tcga_stad)
ZMTfluorouracil_tcga_stad = AutoencoderM(TX_testMfluorouracil_tcga_stad)
ZCTfluorouracil_tcga_stad = AutoencoderC(TX_testCfluorouracil_tcga_stad)
ZTTfluorouracil_tcga_stad = torch.cat((ZETfluorouracil_tcga_stad, ZMTfluorouracil_tcga_stad, ZCTfluorouracil_tcga_stad), 1)
ZTTfluorouracil_tcga_stad = F.normalize(ZTTfluorouracil_tcga_stad, p=2, dim=0)
PredTfluorouracil_tcga_stad = Clas(ZTTfluorouracil_tcga_stad)
AUCTfluorouracil_tcga_stad = roc_auc_score(Ytsfluorouracil_tcga_stad.astype(int), PredTfluorouracil_tcga_stad.detach().numpy())
AUCTfluorouracil_tcga_stad_aucpr = average_precision_score(Ytsfluorouracil_tcga_stad.astype(int), PredTfluorouracil_tcga_stad.detach().numpy())

ZETfluorouracil_tcga_paad = AutoencoderE(TX_testEfluorouracil_tcga_paad)
ZMTfluorouracil_tcga_paad = AutoencoderM(TX_testMfluorouracil_tcga_paad)
ZCTfluorouracil_tcga_paad = AutoencoderC(TX_testCfluorouracil_tcga_paad)
ZTTfluorouracil_tcga_paad = torch.cat((ZETfluorouracil_tcga_paad, ZMTfluorouracil_tcga_paad, ZCTfluorouracil_tcga_paad), 1)
ZTTfluorouracil_tcga_paad = F.normalize(ZTTfluorouracil_tcga_paad, p=2, dim=0)
PredTfluorouracil_tcga_paad = Clas(ZTTfluorouracil_tcga_paad)
AUCTfluorouracil_tcga_paad = roc_auc_score(Ytsfluorouracil_tcga_paad.astype(int), PredTfluorouracil_tcga_paad.detach().numpy())
AUCTfluorouracil_tcga_paad_aucpr = average_precision_score(Ytsfluorouracil_tcga_paad.astype(int), PredTfluorouracil_tcga_paad.detach().numpy())

AUCT_finetuned = dict()
AUCTaucpr_finetuned = dict()
AUCTfluorouracil_pdx_finetuned = dict()
AUCTfluorouracil_tcga_finetuned = dict()
AUCTfluorouracil_pdx_crc_finetuned = dict()
AUCTfluorouracil_tcga_stad_finetuned = dict()
AUCTfluorouracil_tcga_paad_finetuned = dict()
AUCTfluorouracil_pdx_aucpr_finetuned = dict()
AUCTfluorouracil_tcga_aucpr_finetuned = dict()
AUCTfluorouracil_pdx_crc_aucpr_finetuned = dict()
AUCTfluorouracil_tcga_stad_aucpr_finetuned = dict()
AUCTfluorouracil_tcga_paad_aucpr_finetuned = dict()
for iter in range(max_iter):
   iter += 1  

   AutoencoderE_finetuned = torch.load(finetuned_dir + f"Exprs_Fluorouracil_GDSC_PDX_TCGA_DNA_REPi_Expression_Mutation_CNA_Second_Strategy_Finetuned_{iter}.pt")
   AutoencoderM_finetuned = torch.load(finetuned_dir + f"Mut_Fluorouracil_GDSC_PDX_TCGA_DNA_REPi_Expression_Mutation_CNA_Second_Strategy_Finetuned_{iter}.pt")
   AutoencoderC_finetuned = torch.load(finetuned_dir + f"CNA_Fluorouracil_GDSC_PDX_TCGA_DNA_REPi_Expression_Mutation_CNA_Second_Strategy_Finetuned_{iter}.pt")
   Clas_finetuned = torch.load(finetuned_dir + f"Class_Fluorouracil_GDSC_PDX_TCGA_DNA_REPi_Expression_Mutation_CNA_Second_Strategy_Finetuned_{iter}.pt")
   
   AutoencoderE_finetuned.eval()
   AutoencoderM_finetuned.eval()
   AutoencoderC_finetuned.eval()
   Clas_finetuned.eval()
      
   ZEX = AutoencoderE_finetuned(torch.FloatTensor(X_trainE))
   ZMX = AutoencoderM_finetuned(torch.FloatTensor(X_trainM))
   ZCX = AutoencoderC_finetuned(torch.FloatTensor(X_trainC))
   ZTX = torch.cat((ZEX, ZMX, ZCX), 1)
   ZTX = F.normalize(ZTX, p=2, dim=0)
   PredX = Clas_finetuned(ZTX)
   AUCT_finetuned[iter] = roc_auc_score(Y, PredX.detach().numpy())
   AUCTaucpr_finetuned[iter] = average_precision_score(Y, PredX.detach().numpy())
   
   ZETfluorouracil_pdx = AutoencoderE_finetuned(TX_testEfluorouracil_pdx)
   ZMTfluorouracil_pdx = AutoencoderM_finetuned(TX_testMfluorouracil_pdx)
   ZCTfluorouracil_pdx = AutoencoderC_finetuned(TX_testCfluorouracil_pdx)
   ZTTfluorouracil_pdx = torch.cat((ZETfluorouracil_pdx, ZMTfluorouracil_pdx, ZCTfluorouracil_pdx), 1)
   ZTTfluorouracil_pdx = F.normalize(ZTTfluorouracil_pdx, p=2, dim=0)
   PredTfluorouracil_pdx = Clas_finetuned(ZTTfluorouracil_pdx)
   AUCTfluorouracil_pdx_finetuned[iter] = roc_auc_score(Ytsfluorouracil_pdx.astype(int), PredTfluorouracil_pdx.detach().numpy())
   AUCTfluorouracil_pdx_aucpr_finetuned[iter] = average_precision_score(Ytsfluorouracil_pdx.astype(int), PredTfluorouracil_pdx.detach().numpy())

   ZETfluorouracil_pdx_crc = AutoencoderE_finetuned(TX_testEfluorouracil_pdx_crc)
   ZMTfluorouracil_pdx_crc = AutoencoderM_finetuned(TX_testMfluorouracil_pdx_crc)
   ZCTfluorouracil_pdx_crc = AutoencoderC_finetuned(TX_testCfluorouracil_pdx_crc)
   ZTTfluorouracil_pdx_crc = torch.cat((ZETfluorouracil_pdx_crc, ZMTfluorouracil_pdx_crc, ZCTfluorouracil_pdx_crc), 1)
   ZTTfluorouracil_pdx_crc = F.normalize(ZTTfluorouracil_pdx_crc, p=2, dim=0)
   PredTfluorouracil_pdx_crc = Clas_finetuned(ZTTfluorouracil_pdx_crc)
   AUCTfluorouracil_pdx_crc_finetuned[iter] = roc_auc_score(Ytsfluorouracil_pdx_crc.astype(int), PredTfluorouracil_pdx_crc.detach().numpy())
   AUCTfluorouracil_pdx_crc_aucpr_finetuned[iter] = average_precision_score(Ytsfluorouracil_pdx_crc.astype(int), PredTfluorouracil_pdx_crc.detach().numpy())

   ZETfluorouracil_tcga = AutoencoderE_finetuned(TX_testEfluorouracil_tcga)
   ZMTfluorouracil_tcga = AutoencoderM_finetuned(TX_testMfluorouracil_tcga)
   ZCTfluorouracil_tcga = AutoencoderC_finetuned(TX_testCfluorouracil_tcga)
   ZTTfluorouracil_tcga = torch.cat((ZETfluorouracil_tcga, ZMTfluorouracil_tcga, ZCTfluorouracil_tcga), 1)
   ZTTfluorouracil_tcga = F.normalize(ZTTfluorouracil_tcga, p=2, dim=0)
   PredTfluorouracil_tcga = Clas_finetuned(ZTTfluorouracil_tcga)
   AUCTfluorouracil_tcga_finetuned[iter] = roc_auc_score(Ytsfluorouracil_tcga.astype(int), PredTfluorouracil_tcga.detach().numpy())
   AUCTfluorouracil_tcga_aucpr_finetuned[iter] = average_precision_score(Ytsfluorouracil_tcga.astype(int), PredTfluorouracil_tcga.detach().numpy())

   ZETfluorouracil_tcga_stad = AutoencoderE_finetuned(TX_testEfluorouracil_tcga_stad)
   ZMTfluorouracil_tcga_stad = AutoencoderM_finetuned(TX_testMfluorouracil_tcga_stad)
   ZCTfluorouracil_tcga_stad = AutoencoderC_finetuned(TX_testCfluorouracil_tcga_stad)
   ZTTfluorouracil_tcga_stad = torch.cat((ZETfluorouracil_tcga_stad, ZMTfluorouracil_tcga_stad, ZCTfluorouracil_tcga_stad), 1)
   ZTTfluorouracil_tcga_stad = F.normalize(ZTTfluorouracil_tcga_stad, p=2, dim=0)
   PredTfluorouracil_tcga_stad = Clas_finetuned(ZTTfluorouracil_tcga_stad)
   AUCTfluorouracil_tcga_stad_finetuned[iter] = roc_auc_score(Ytsfluorouracil_tcga_stad.astype(int), PredTfluorouracil_tcga_stad.detach().numpy())
   AUCTfluorouracil_tcga_stad_aucpr_finetuned[iter] = average_precision_score(Ytsfluorouracil_tcga_stad.astype(int), PredTfluorouracil_tcga_stad.detach().numpy())

   ZETfluorouracil_tcga_paad = AutoencoderE_finetuned(TX_testEfluorouracil_tcga_paad)
   ZMTfluorouracil_tcga_paad = AutoencoderM_finetuned(TX_testMfluorouracil_tcga_paad)
   ZCTfluorouracil_tcga_paad = AutoencoderC_finetuned(TX_testCfluorouracil_tcga_paad)
   ZTTfluorouracil_tcga_paad = torch.cat((ZETfluorouracil_tcga_paad, ZMTfluorouracil_tcga_paad, ZCTfluorouracil_tcga_paad), 1)
   ZTTfluorouracil_tcga_paad = F.normalize(ZTTfluorouracil_tcga_paad, p=2, dim=0)
   PredTfluorouracil_tcga_paad = Clas_finetuned(ZTTfluorouracil_tcga_paad)
   AUCTfluorouracil_tcga_paad_finetuned[iter] = roc_auc_score(Ytsfluorouracil_tcga_paad.astype(int), PredTfluorouracil_tcga_paad.detach().numpy())
   AUCTfluorouracil_tcga_paad_aucpr_finetuned[iter] = average_precision_score(Ytsfluorouracil_tcga_paad.astype(int), PredTfluorouracil_tcga_paad.detach().numpy())

max_key_pdx = max(AUCTfluorouracil_pdx_aucpr_finetuned, key=AUCTfluorouracil_pdx_aucpr_finetuned.get)
max_key_pdx_crc = max(AUCTfluorouracil_pdx_crc_aucpr_finetuned, key=AUCTfluorouracil_pdx_crc_aucpr_finetuned.get)
max_key_tcga = max(AUCTfluorouracil_tcga_aucpr_finetuned, key=AUCTfluorouracil_tcga_aucpr_finetuned.get)
max_key_tcga_stad = max(AUCTfluorouracil_tcga_stad_aucpr_finetuned, key=AUCTfluorouracil_tcga_stad_aucpr_finetuned.get)
max_key_tcga_paad = max(AUCTfluorouracil_tcga_paad_aucpr_finetuned, key=AUCTfluorouracil_tcga_paad_aucpr_finetuned.get)

print(f"GDSC - Training AUC (DNA_REPi): {round(AUCT,2)}")
print(f"GDSC - Training AUCPR (DNA_REPi): {round(AUCTaucpr,2)}")
print(f"PDX - Test AUC (5-Fluorouracil): {round(AUCTfluorouracil_pdx,2)}")
print(f"PDX - Test AUCPR (5-Fluorouracil): {round(AUCTfluorouracil_pdx_aucpr,2)}")
print(f"PDX - Test AUC (5-Fluorouracil, CRC): {round(AUCTfluorouracil_pdx_crc,2)}")
print(f"PDX - Test AUCPR (5-Fluorouracil, CRC): {round(AUCTfluorouracil_pdx_crc_aucpr,2)}")
print(f"TCGA - Test AUC (5-Fluorouracil): {round(AUCTfluorouracil_tcga,2)}")
print(f"TCGA - Test AUCPR (5-Fluorouracil): {round(AUCTfluorouracil_tcga_aucpr,2)}")
print(f"TCGA - Test AUC (5-Fluorouracil, STAD): {round(AUCTfluorouracil_tcga_stad,2)}")
print(f"TCGA - Test AUCPR (5-Fluorouracil, STAD): {round(AUCTfluorouracil_tcga_stad_aucpr,2)}")
print(f"TCGA - Test AUC (5-Fluorouracil, PAAD): {round(AUCTfluorouracil_tcga_paad,2)}")
print(f"TCGA - Test AUCPR (5-Fluorouracil, PAAD): {round(AUCTfluorouracil_tcga_paad_aucpr,2)}\n")

print(f"GDSC - Training AUC (DNA_REPi, Finetuned): {round(AUCT_finetuned[max_key_pdx],2)}")
print(f"GDSC - Training AUCPR (DNA_REPi, Finetuned): {round(AUCTaucpr_finetuned[max_key_pdx],2)}")
print(f"PDX - Test AUC (5-Fluorouracil, Finetuned): {round(AUCTfluorouracil_pdx_finetuned[max_key_pdx],2)}")
print(f"PDX - Test AUCPR (5-Fluorouracil, Finetuned): {round(AUCTfluorouracil_pdx_aucpr_finetuned[max_key_pdx],2)}\n")

print(f"GDSC - Training AUC (DNA_REPi, Finetuned): {round(AUCT_finetuned[max_key_pdx_crc],2)}")
print(f"GDSC - Training AUCPR (DNA_REPi, Finetuned): {round(AUCTaucpr_finetuned[max_key_pdx_crc],2)}")
print(f"PDX - Test AUC (5-Fluorouracil, CRC, Finetuned): {round(AUCTfluorouracil_pdx_crc_finetuned[max_key_pdx_crc],2)}")
print(f"PDX - Test AUCPR (5-Fluorouracil, CRC, Finetuned): {round(AUCTfluorouracil_pdx_crc_aucpr_finetuned[max_key_pdx_crc],2)}\n")

print(f"GDSC - Training AUC (DNA_REPi, Finetuned): {round(AUCT_finetuned[max_key_tcga],2)}")
print(f"GDSC - Training AUCPR (DNA_REPi, Finetuned): {round(AUCTaucpr_finetuned[max_key_tcga],2)}")
print(f"TCGA - Test AUC (5-Fluorouracil, Finetuned): {round(AUCTfluorouracil_tcga_finetuned[max_key_tcga],2)}")
print(f"TCGA - Test AUCPR (5-Fluorouracil, Finetuned): {round(AUCTfluorouracil_tcga_aucpr_finetuned[max_key_tcga],2)}\n")

print(f"GDSC - Training AUC (DNA_REPi, Finetuned): {round(AUCT_finetuned[max_key_tcga_stad],2)}")
print(f"GDSC - Training AUCPR (DNA_REPi, Finetuned): {round(AUCTaucpr_finetuned[max_key_tcga_stad],2)}")
print(f"TCGA - Test AUC (5-Fluorouracil, STAD, Finetuned): {round(AUCTfluorouracil_tcga_stad_finetuned[max_key_tcga_stad],2)}")
print(f"TCGA - Test AUCPR (5-Fluorouracil, STAD, Finetuned): {round(AUCTfluorouracil_tcga_stad_aucpr_finetuned[max_key_tcga_stad],2)}\n")

print(f"GDSC - Training AUC (DNA_REPi, Finetuned): {round(AUCT_finetuned[max_key_tcga_paad],2)}")
print(f"GDSC - Training AUCPR (DNA_REPi, Finetuned): {round(AUCTaucpr_finetuned[max_key_tcga_paad],2)}")
print(f"TCGA - Test AUC (5-Fluorouracil, PAAD, Finetuned): {round(AUCTfluorouracil_tcga_paad_finetuned[max_key_tcga_paad],2)}")
print(f"TCGA - Test AUCPR (5-Fluorouracil, PAAD, Finetuned): {round(AUCTfluorouracil_tcga_paad_aucpr_finetuned[max_key_tcga_paad],2)}\n\n")

print("Models for maximum finetuning (5-Fluorouracil, PDX)")
print(f"Exprs_Fluorouracil_GDSC_PDX_TCGA_DNA_REPi_Expression_Mutation_CNA_Second_Strategy_Finetuned_{max_key_pdx}.pt")
print(f"Mut_Fluorouracil_GDSC_PDX_TCGA_DNA_REPi_Expression_Mutation_CNA_Second_Strategy_Finetuned_{max_key_pdx}.pt")
print(f"CNA_Fluorouracil_GDSC_PDX_TCGA_DNA_REPi_Expression_Mutation_CNA_Second_Strategy_Finetuned_{max_key_pdx}.pt")
print(f"Class_Fluorouracil_GDSC_PDX_TCGA_DNA_REPi_Expression_Mutation_CNA_Second_Strategy_Finetuned_{max_key_pdx}.pt")

print("\nModels for maximum finetuning (5-Fluorouracil, PDX, CRC)")
print(f"Exprs_Fluorouracil_GDSC_PDX_TCGA_DNA_REPi_Expression_Mutation_CNA_Second_Strategy_Finetuned_{max_key_pdx_crc}.pt")
print(f"Mut_Fluorouracil_GDSC_PDX_TCGA_DNA_REPi_Expression_Mutation_CNA_Second_Strategy_Finetuned_{max_key_pdx_crc}.pt")
print(f"CNA_Fluorouracil_GDSC_PDX_TCGA_DNA_REPi_Expression_Mutation_CNA_Second_Strategy_Finetuned_{max_key_pdx_crc}.pt")
print(f"Class_Fluorouracil_GDSC_PDX_TCGA_DNA_REPi_Expression_Mutation_CNA_Second_Strategy_Finetuned_{max_key_pdx_crc}.pt")

print("\nModels for maximum finetuning (5-Fluorouracil, TCGA)")
print(f"Exprs_Fluorouracil_GDSC_PDX_TCGA_DNA_REPi_Expression_Mutation_CNA_Second_Strategy_Finetuned_{max_key_tcga}.pt")
print(f"Mut_Fluorouracil_GDSC_PDX_TCGA_DNA_REPi_Expression_Mutation_CNA_Second_Strategy_Finetuned_{max_key_tcga}.pt")
print(f"CNA_Fluorouracil_GDSC_PDX_TCGA_DNA_REPi_Expression_Mutation_CNA_Second_Strategy_Finetuned_{max_key_tcga}.pt")
print(f"Class_Fluorouracil_GDSC_PDX_TCGA_DNA_REPi_Expression_Mutation_CNA_Second_Strategy_Finetuned_{max_key_tcga}.pt")

print("\nModels for maximum finetuning (5-Fluorouracil, TCGA, STAD)")
print(f"Exprs_Fluorouracil_GDSC_PDX_TCGA_DNA_REPi_Expression_Mutation_CNA_Second_Strategy_Finetuned_{max_key_tcga_stad}.pt")
print(f"Mut_Fluorouracil_GDSC_PDX_TCGA_DNA_REPi_Expression_Mutation_CNA_Second_Strategy_Finetuned_{max_key_tcga_stad}.pt")
print(f"CNA_Fluorouracil_GDSC_PDX_TCGA_DNA_REPi_Expression_Mutation_CNA_Second_Strategy_Finetuned_{max_key_tcga_stad}.pt")
print(f"Class_Fluorouracil_GDSC_PDX_TCGA_DNA_REPi_Expression_Mutation_CNA_Second_Strategy_Finetuned_{max_key_tcga_stad}.pt")

print("\nModels for maximum finetuning (5-Fluorouracil, TCGA, PAAD)")
print(f"Exprs_Fluorouracil_GDSC_PDX_TCGA_DNA_REPi_Expression_Mutation_CNA_Second_Strategy_Finetuned_{max_key_tcga_paad}.pt")
print(f"Mut_Fluorouracil_GDSC_PDX_TCGA_DNA_REPi_Expression_Mutation_CNA_Second_Strategy_Finetuned_{max_key_tcga_paad}.pt")
print(f"CNA_Fluorouracil_GDSC_PDX_TCGA_DNA_REPi_Expression_Mutation_CNA_Second_Strategy_Finetuned_{max_key_tcga_paad}.pt")
print(f"Class_Fluorouracil_GDSC_PDX_TCGA_DNA_REPi_Expression_Mutation_CNA_Second_Strategy_Finetuned_{max_key_tcga_paad}.pt")


GDSC - Training AUC (DNA_REPi): 0.81
GDSC - Training AUCPR (DNA_REPi): 0.34
PDX - Test AUC (5-Fluorouracil): 0.14
PDX - Test AUCPR (5-Fluorouracil): 0.05
PDX - Test AUC (5-Fluorouracil, CRC): 0.14
PDX - Test AUCPR (5-Fluorouracil, CRC): 0.05
TCGA - Test AUC (5-Fluorouracil): 0.53
TCGA - Test AUCPR (5-Fluorouracil): 0.74
TCGA - Test AUC (5-Fluorouracil, STAD): 0.75
TCGA - Test AUCPR (5-Fluorouracil, STAD): 0.89
TCGA - Test AUC (5-Fluorouracil, PAAD): 0.25
TCGA - Test AUCPR (5-Fluorouracil, PAAD): 0.32

GDSC - Training AUC (DNA_REPi, Finetuned): 0.81
GDSC - Training AUCPR (DNA_REPi, Finetuned): 0.34
PDX - Test AUC (5-Fluorouracil, Finetuned): 0.18
PDX - Test AUCPR (5-Fluorouracil, Finetuned): 0.05

GDSC - Training AUC (DNA_REPi, Finetuned): 0.81
GDSC - Training AUCPR (DNA_REPi, Finetuned): 0.34
PDX - Test AUC (5-Fluorouracil, CRC, Finetuned): 0.18
PDX - Test AUCPR (5-Fluorouracil, CRC, Finetuned): 0.05

GDSC - Training AUC (DNA_REPi, Finetuned): 0.81
GDSC - Training AUCPR (DNA_REPi, Fine

In [73]:
# show expression layer parameters

AutoencoderE

AEE(
  (EnE): Sequential(
    (0): Linear(in_features=631, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
  )
)

In [74]:
# show mutation layer parameters

AutoencoderM

AEM(
  (EnM): Sequential(
    (0): Linear(in_features=631, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.7, inplace=False)
  )
)

In [75]:
# show CNA layer parameters

AutoencoderC

AEC(
  (EnC): Sequential(
    (0): Linear(in_features=631, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
  )
)

In [76]:
# show classification layer parameters

Clas

Classifier(
  (FC): Sequential(
    (0): Linear(in_features=1408, out_features=1, bias=True)
    (1): Dropout(p=0.3, inplace=False)
    (2): Sigmoid()
  )
)

In [77]:
# create a dataframe with the prediction results 

emc  = {"Data": ["Expr + Mut + CNA", "Expr + Mut + CNA (Fine-tuned)"],
       "AUC (GDSC)": [round(AUCT, 2), round(AUCT_finetuned[max_key_pdx], 2)],
       "AUCPR (GDSC)": [round(AUCTaucpr,2), round(AUCTaucpr_finetuned[max_key_pdx],2)],
       "AUC (PDX)": [round(AUCTfluorouracil_pdx,2), round(AUCTfluorouracil_pdx_finetuned[max_key_pdx],2)],
       "AUCPR (PDX)":  [round(AUCTfluorouracil_pdx_aucpr,2), round(AUCTfluorouracil_pdx_aucpr_finetuned[max_key_pdx],2)],
       "AUC (PDX-CRC)":  [round(AUCTfluorouracil_pdx_crc,2), round(AUCTfluorouracil_pdx_crc_finetuned[max_key_pdx_crc],2)],
       "AUCPR (PDX-CRC)": [round(AUCTfluorouracil_pdx_crc_aucpr,2), round(AUCTfluorouracil_pdx_crc_aucpr_finetuned[max_key_pdx_crc],2)],
       "AUC (TCGA)": [round(AUCTfluorouracil_tcga,2), round(AUCTfluorouracil_tcga_finetuned[max_key_tcga],2)],
       "AUCPR (TCGA)":  [round(AUCTfluorouracil_tcga_aucpr,2), round(AUCTfluorouracil_tcga_aucpr_finetuned[max_key_tcga],2)],
       "AUC (TCGA-STAD)":  [round(AUCTfluorouracil_tcga_stad,2), round(AUCTfluorouracil_tcga_stad_finetuned[max_key_tcga_stad],2)],
       "AUCPR (TCGA-STAD)": [round(AUCTfluorouracil_tcga_stad_aucpr,2), round(AUCTfluorouracil_tcga_stad_aucpr_finetuned[max_key_tcga_stad],2)],
       "AUC (TCGA-PAAD)":  [round(AUCTfluorouracil_tcga_paad,2), round(AUCTfluorouracil_tcga_paad_finetuned[max_key_tcga_paad],2)],
       "AUCPR (TCGA-PAAD)": [round(AUCTfluorouracil_tcga_paad_aucpr,2), round(AUCTfluorouracil_tcga_paad_aucpr_finetuned[max_key_tcga_paad],2)],
       "Sample Size (GDSC)": [GDSCEv2.shape[0], GDSCEv2.shape[0]],
       "Feature Size (GDSC)": [GDSCEv2.shape[1], GDSCEv2.shape[1]] 
}

emc_dataframe = pd.DataFrame.from_dict(emc, orient='index').transpose() 
os.makedirs(save_results_to, exist_ok = True)
emc_dataframe.to_csv(save_results_to + "GDSC_PDX_TCGA_Expression_Mutation_CNA_DNA_REPi_Fluorouracil_Second_Strategy.tsv",
                     sep = "\t",
                     index = False)

emc_dataframe

,Data,AUC (GDSC),AUCPR (GDSC),AUC (PDX),AUCPR (PDX),AUC (PDX-CRC),AUCPR (PDX-CRC),AUC (TCGA),AUCPR (TCGA),AUC (TCGA-STAD),AUCPR (TCGA-STAD),AUC (TCGA-PAAD),AUCPR (TCGA-PAAD),Sample Size (GDSC),Feature Size (GDSC)
0,Expr + Mut + CNA,0.81,0.34,0.14,0.05,0.14,0.05,0.53,0.74,0.75,0.89,0.25,0.32,9435,631
1,Expr + Mut + CNA (Fine-tuned),0.81,0.34,0.18,0.05,0.18,0.05,0.57,0.78,0.74,0.9,0.5,0.45,9435,631


In [78]:
# change directory

os.chdir(save_results_to)
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/Transfer Learning/Drugs with same pathways/DNA_REPi/Expression_Mutation_CNA/Predictions/GDSC_PDX_TCGA_Second_Strategy


In [79]:
# save predictions

file = open('GDSC_PDX_TCGA_DNA_REPi_Fluorouracil_Predictions_Second_Strategy (Expression, Mutation, and CNA).txt', 'w')
file.write(f"GDSC Training (EMC) AUC (DNA_REPi): {round(AUCT,2)}\n")
file.write(f"GDSC Training (EMC) AUCPR (DNA_REPi): {round(AUCTaucpr,2)}\n\n")

file.write(f"PDX Test (EMC) AUC (5-Fluorouracil): {round(AUCTfluorouracil_pdx,2)}\n")
file.write(f"PDX Test (EMC) AUCPR (5-Fluorouracil): {round(AUCTfluorouracil_pdx_aucpr,2)}\n\n")

file.write(f"PDX Test (EMC) AUC (5-Fluorouracil, CRC): {round(AUCTfluorouracil_pdx_crc,2)}\n")
file.write(f"PDX Test (EMC) AUCPR (5-Fluorouracil, CRC): {round(AUCTfluorouracil_pdx_crc_aucpr,2)}\n\n")

file.write(f"TCGA Test (EMC) AUC (5-Fluorouracil): {round(AUCTfluorouracil_tcga,2)}\n")
file.write(f"TCGA Test (EMC) AUCPR (5-Fluorouracil): {round(AUCTfluorouracil_tcga_aucpr,2)}\n\n")

file.write(f"TCGA Test (EMC) AUC (5-Fluorouracil, STAD): {round(AUCTfluorouracil_tcga_stad,2)}\n")
file.write(f"TCGA Test (EMC) AUCPR (5-Fluorouracil, STAD): {round(AUCTfluorouracil_tcga_stad_aucpr,2)}\n")

file.write(f"TCGA Test (EMC) AUC (5-Fluorouracil, PAAD): {round(AUCTfluorouracil_tcga_paad,2)}\n")
file.write(f"TCGA Test (EMC) AUCPR (5-Fluorouracil, PAAD): {round(AUCTfluorouracil_tcga_paad_aucpr,2)}\n")

file.close()